# Layouts

With layouts, meta data of an HDF5 file can be defined.

In [ ]:
import h5rdmtoolbox as h5tbx

Initialize a `H5FileLayout`:

In [ ]:
h5tbx.generate_temporary_filename(suffix='.hdf')

In [ ]:
mylayout = h5tbx.conventions.layout.Layout(h5tbx.generate_temporary_filename(suffix='.hdf'))

Say we expect HDF files to have the following data and structure:
 - a root attribtue "title", which an have any value
 - the datasets "x" and "y" which are 1d-arrays, which must have the attributes "standard_name" or "long_name"
 - a group "meta"

Fill the file with content. Note, the layout file is an HDF file:

In [ ]:
with mylayout.File(mode='w') as h5:
    h5.attrs['title'] = '__any'
    dsx = h5.create_dataset('x', shape=(1,))  # shape does not matter here, but has to be passed
    dsx.attrs['standard_name.alt:long_name'] = '__any'
    dsy = h5.create_dataset('y', shape=(1,))  # shape does not matter here, but has to be passed
    dsy.attrs['standard_name.alt:long_name'] = '__any'
    h5.create_group('meta')
    h5.create_group('meta/other')
mylayout.dump()

Now let's create an example HDF5 file and check if the layout is ok or if there are issues:

In [ ]:
import h5py
test_filename = h5tbx.generate_temporary_filename()
with h5py.File(test_filename, mode='w') as h5:
    h5.attrs['title'] = 'my test file'
    dsx = h5.create_dataset('x', data=[1,2,3])
    dsx.attrs['long_name'] = 'x coordinate'
    dsx.attrs['standard_name'] = 'x_coordinate'
    dsy = h5.create_dataset('y', data=[[1,2],[3,4]])

We use `check()` and pass the opened hdf5 group to check for any issues. We already know, taht the group "meta" and "meta/other" are missing, as well the attribute "standard_name" or "long_name" for dataset "y". Note, that missing group "other" will not raise an issue because "meta" raised one before.

In [ ]:
with h5py.File(test_filename, mode='r') as h5:
    mylayout.check(h5, silent=True)
print(mylayout)

Note, that we can enter the check-process at any level. However, no check is performed above that group, in this case the group "meta" which we add before we run a check again beginning from that group level:

In [ ]:
with h5py.File(test_filename, mode='r+') as h5:
    h5.create_group('meta')
    mylayout.check(h5['meta'], silent=True)
print(mylayout)

Only checking the specified group requires to pass `recursive=False`:

In [ ]:
with h5py.File(test_filename, mode='r+') as h5:
    mylayout.check(h5['/'], silent=True, recursive=False)
print(mylayout)